# Import packages

In [ ]:
!pip install torcheval onnx wandb kaggle --quiet

In [1]:
import os
import json
import numpy as np
import pandas as pd
import torch
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split, WeightedRandomSampler
import torch.optim as optim
from torcheval.metrics.functional import multiclass_f1_score
import wandb

# Download data if not found
### *You should upload kaggle.json to the current working dir before running this cell*

In [2]:
if not os.path.exists("Data"):
    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !ls ~/.kaggle
    !chmod 600 ~/.kaggle/kaggle.json  # set permission
    !kaggle competitions download -c cassava-leaf-disease-classification -p ./
    !mkdir Data
    !unzip -q -n cassava-leaf-disease-classification.zip -d ./Data
    !rm cassava-leaf-disease-classification.zip

# Wandb Setup
### *Change the env below to be your notebook path, also follow the prompt to enter your wandb token*

In [3]:
os.environ['WANDB_NOTEBOOK_NAME'] = "MajorityPredict.ipynb"
wandb.login()

wandb: Currently logged in as: wyq (cassava). Use `wandb login --relogin` to force relogin


True

# *Define Customized Dataset*

In [4]:
class LeafDataset(Dataset):
    def __init__(self, root_path, transform=None):
        self.image_path = root_path + '/train_images'
        self.labels = pd.read_csv(root_path + '/train.csv')
        self.transform = transform

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.image_path, self.labels['image_id'][idx])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        if "label" in self.labels.columns:
            label = self.labels['label'][idx]
            sample = (image, label)
        else:
            sample = (image)
        return sample

# Define Torch Module
### *Change this to your own model*

In [5]:
class MajorityPredict(nn.Module):
    def __init__(self, n_inputs, n_outputs):
        super().__init__()

    def forward(self, x):
        batch = x.shape[0]
        # Perform the forward pass through the layers
        x = torch.tensor([0.0, 0.0, 0.0, 1.0, 0.0]).cuda()
        x = x.expand((batch, 5))
        return x

# Define Run Segments

## The Whole Pipeline

In [6]:
def model_pipeline(hyperparameters):
    # tell wandb to get started
    with wandb.init(project="cassava-leaf", config=hyperparameters):
        # access all HPs through wandb.config, so logging matches execution!
        config = wandb.config

        # make the model, data, and optimization problem
        model, train_loader, eval_loader, test_loader, criterion, optimizer = make(config)
        print(model)

        # and use them to train the model
        train(model, train_loader, eval_loader, criterion, optimizer, config)

        # and test its final performance
        test(model, test_loader)

    return model

## 1. Make (Model, loaders...)

### Helper functions for Make

In [7]:
def get_datasets(split, transform):
    total_dataset = LeafDataset('./Data', transform=transform)
    subsets = random_split(total_dataset,
                           split,
                           generator=torch.Generator().manual_seed(42))
    return subsets


def make_loaders(datasets, batch_size, num_workers, balance=False):
    if balance:
        # compute class weights:
        class_weights = [0] * 5
        for _, label in datasets[0]:
            class_weights[label] += 1
        class_weights = [10000 / i for i in class_weights]

        # compute sample weights
        sample_weights = [0] * len(datasets[0])
        for idx, (data, label) in enumerate(datasets[0]):
            sample_weights[idx] = class_weights[label]
        # init weighted sampler
        sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

        train_dataloader = DataLoader(datasets[0], batch_size=batch_size, num_workers=num_workers, sampler=sampler)
    else:
        train_dataloader = DataLoader(datasets[0], batch_size=batch_size, shuffle=True, num_workers=num_workers)
    eval_dataloader = DataLoader(datasets[1], batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_dataloader = DataLoader(datasets[2], batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return train_dataloader, eval_dataloader, test_dataloader

### Make main function (Adjust Transforms / Model Declaration Here)

In [8]:
def make(config):
    transform = transforms.Compose([
        transforms.Resize((config.resize, config.resize)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])

    # Make the data
    datasets = get_datasets(config.split, transform)
    train_loader, eval_loader, test_loader = make_loaders(datasets,
                                                          batch_size=config.batch_size,
                                                          num_workers=config.num_workers,
                                                          balance=config.balance)

    # Make the model
    model = MajorityPredict(3 * config.resize * config.resize, 5)
    model.cuda()

    # Make the loss and optimizer
    criterion = None
    optimizer = None
    return model, train_loader, eval_loader, test_loader, criterion, optimizer

## 2. Train

### Helper functions for Train

In [9]:
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.
    correct, total = 0, 0
    for _, (images, labels) in enumerate(train_loader):
        # move data
        images = images.cuda()
        labels = labels.cuda()

        # forward prop
        output = model(images)
        loss = criterion(output, labels)

        # backward prop
        optimizer.zero_grad()
        loss.backward()

        # update weights
        optimizer.step()

        # record performance
        running_loss += loss.item()
        _, predicted = torch.max(output, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    train_loss = running_loss / len(train_loader)
    train_acc = correct / total
    return train_loss, train_acc


def eval_epoch(model, eval_loader, criterion):
    model.eval()
    running_loss = 0.
    correct, total = 0, 0
    for _, (images, labels) in enumerate(eval_loader):
        # move data
        images = images.cuda()
        labels = labels.cuda()

        # forward prop
        output = model(images)
        loss = criterion(output, labels)

        # record performance
        running_loss += loss.item()
        _, predicted = torch.max(output, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    eval_loss = running_loss / len(eval_loader)
    eval_acc = correct / total
    return eval_loss, eval_acc

### Train main function

In [10]:
def train(model, train_loader, eval_loader, criterion, optimizer, config):
    wandb.watch(model, criterion, log="all", log_freq=10)
    for epoch in range(config.epochs):

        # train model
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
        # evaluate model
        eval_loss, eval_acc = eval_epoch(model, eval_loader, criterion)

        # log to wandb
        wandb.log({"train_loss": train_loss,
                   "train_acc": train_acc,
                   "eval_loss": eval_loss,
                   "eval_acc": eval_acc}, step=epoch)
        print(f"Epoch {epoch}: Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.3f}, Eval Loss: {eval_loss:.3f}, Eval Acc: {eval_acc:.3f}")

## 3. Test

In [11]:
def test(model, test_loader):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        label_total = torch.tensor([])
        pred_total = torch.tensor([])
        for images, labels in test_loader:
            images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            # compute correct / total samples
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # concate labels
            label_total = torch.cat((label_total, labels.detach().cpu()), dim=0)
            pred_total = torch.cat((pred_total, predicted.detach().cpu()), dim=0)
        # compute accuracy
        acc = correct / total
        wandb.log({"test_accuracy": acc})
        # compute f1 scores
        pred_total = pred_total.to(torch.int64)
        label_total = label_total.to(torch.int64)
        f1_micro = multiclass_f1_score(pred_total,
                                       label_total,
                                       num_classes=5,
                                       average="micro")
        f1_macro = multiclass_f1_score(pred_total,
                                       label_total,
                                       num_classes=5,
                                       average="macro")
        f1_each = multiclass_f1_score(pred_total,
                                      label_total,
                                      num_classes=5,
                                      average=None)
        print(f"Test Acc: {acc:.3f}, F1 micro: {f1_micro:.3f}, F1 macro: {f1_macro:.3f}, F1 each: {f1_each}")
        f1 = [[f"f1_class_{idx}", value] for idx, value in enumerate(f1_each)]
        f1.append(["f1_micro", f1_micro])
        f1.append(["f1_macro", f1_macro])
        table = wandb.Table(data=f1, columns=["class", "f1_score"])
        wandb.log({"my_bar_chart_1": wandb.plot.bar(table, "class", "f1_score", title="F1 Score")})

        torch.save(model.state_dict(), "model_state.pth")
        wandb.save("model_state.pth")
        torch.onnx.export(model, images, "model.onnx")
        wandb.save("model.onnx")

# Run Pipeline
# Define HyperParameters
### *Change this according to your run, add config if needed*

In [12]:
config = dict(
    model="MajorityPredict",
    split=[0.4, 0.2, 0.4],  # train / val / test split ratio
    batch_size=256,
    resize=224,
    num_workers=2,  # number of workers per dataloader
    balance=False,  # weight balance train dataset
    epochs=0,
    learning_rate=0.01,
    weight_decay=0.01,  # L2 regularization hyperparamter for Adam Optimizer
)

model = model_pipeline(config)

MajorityPredict()
Test Acc: 0.620, F1 micro: 0.620, F1 macro: 0.153, F1 each: tensor([0.0000, 0.0000, 0.0000, 0.7657, 0.0000])


/tmp/ipykernel_27481/595700726.py:8: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  x = torch.tensor([0.0, 0.0, 0.0, 1.0, 0.0]).cuda()


test_accuracy,▁
test_accuracy,0.62036
